# Building Model

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.feature_selection import SelectKBest, chi2, f_regression, f_classif

In [3]:
# creating Path object for current working directory
directory = Path('./')
# creating Path object for additional data directory
additional_directory = directory / 'additional_data'
# create new directory for additional data
Path(additional_directory).mkdir(exist_ok=True)

# defining the directory to original data
directory = Path('./data/')
additional_directory = Path('./additional_data')

# list the .csv files for the project
for file in directory.glob('*.csv'):
    print(file)
    
# reading in .csv files to dataframes
vehicles = pd.read_csv(directory / 'vehicles2019.csv', dtype={'Accident_Index': str})
casualties = pd.read_csv(directory / 'casualties2019.csv', dtype={'Accident_Index': str})
# cleaned accidents DataFrame
accidents = pd.read_pickle('accidents_cleaned.pkl')

# convert column names to lowercase for ease of indexing
def lower_columns(df):
    """
    Defintion:
        convert column names to lower case
    """
    df.columns = map(str.lower, df.columns)
    
# converting all column names to lower case
lower_columns(vehicles)
lower_columns(casualties)

accidents.head(5)

data/vehicles2019.csv
data/accidents2019.csv
data/casualties2019.csv


accident_index  longitude   latitude  police_force  accident_severity  \
0  2019010152270  -0.127949  51.436208             1                  3   
1  2019010157567  -0.123427  51.449310             1                  3   
2  2019010157732  -0.145106  51.461256             1                  2   
3  2019010157896  -0.240823  51.533125             1                  3   
4  2019010157795   0.161736  51.550272             1                  3   

   number_of_vehicles  number_of_casualties  day_of_week  \
0                   2                     1            3   
1                   2                     2            3   
2                   1                     2            3   
3                   2                     1            3   
4                   1                     1            3   

   local_authority_(district) local_authority_(highway)  ...  \
0                           9                 E09000022  ...   
1                           9                 E09000022  ...   
2                           9                 E09000022  ...   
3                          28                 E09000005  ...   
4                          16                 E09000002  ...   

   carriageway_hazards  urban_or_rural_area  \
0                    0                    1   
1                    0                    1   
2                    0                    1   
3                    0                    1   
4                    0                    1   

   did_police_officer_attend_scene_of_accident  lsoa_of_accident_location  \
0                                            3                  E01003117   
1                                            1                  E01003023   
2                                            1                  E01003026   
3                                            1                  E01000535   
4                                            1                  E01000035   

               district  converted_date  converted_time            datetime  \
0               lambeth      2019-01-15        21:45:00 2019-01-15 21:45:00   
1               lambeth      2019-01-15        08:42:00 2019-01-15 08:42:00   
2               lambeth      2019-01-15        07:08:00 2019-01-15 07:08:00   
3                 brent      2019-01-15        21:05:00 2019-01-15 21:05:00   
4  barking and dagenham      2019-01-15        16:10:00 2019-01-15 16:10:00   

   decimal_time  day_of_year  
0     21.750000           15  
1      8.700000           15  
2      7.133333           15  
3     21.083333           15  
4     16.166667           15  

[5 rows x 34 columns]

In [4]:
predictors = ['weather_conditions', 'speed_limit',
              'road_surface_conditions', 'light_conditions']

usethis = accidents.dropna()
usethis.reset_index(drop=True)

usethis = usethis.filter(predictors, axis=1)

wc_pos_mask = usethis.weather_conditions > 0
sl_pos_mask = usethis.speed_limit > 0
rsc_pos_mask = usethis.road_surface_conditions > 0
lc_pos_mask = usethis.light_conditions > 0

usethisnow = usethis.loc[wc_pos_mask & sl_pos_mask & rsc_pos_mask & lc_pos_mask, :]

In [5]:
selector = SelectKBest(f_classif, k='all')
selector.fit(usethisnow[predictors], usethisnow['accident_severity'])

scores = -np.log(selector.pvalues_)

KeyError: 'accident_severity'